# Imports

In [ ]:
# Numpy, Scipy, Matplotlib
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
import numpy as np

# Drake imports
import pydrake
from pydrake.all import (
    DirectCollocation, DirectTranscription, MathematicalProgram,
    InputPortSelection, LogVectorOutput
)
from pydrake.all import FindResourceOrThrow
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex,
                        RotationMatrix, Meshcat,MeshcatVisualizerParams, MeshcatVisualizerCpp, MeshcatVisualizer,
                        Adder, Gain, ConstantVectorSource, Demultiplexer, Multiplexer, AngleAxis)

# Other imports
import importlib
import re
import enum
from IPython.display import display, SVG, Image
import pydot

# Imports of other project files
import constants
import config
from config import hinge_rotation_axis

import plant.simulation
import plant.manipulator as manipulator


import ctrl.aux


In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# default_figsize = (2*3,2*2)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)
default_figsize = (16,8)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16,8)
matplotlib.rcParams['lines.linewidth'] = 5
matplotlib.rcParams["axes.xmargin"] = 0

# Meshcat init

In [ ]:
meshcat = Meshcat()
web_url = meshcat.web_url()

# Simulation setup

In [ ]:
ctrl_paradigm = plant.simulation.CtrlParadigm.IMPEDANCE
impedance_type = plant.simulation.ImpedanceType.LINK_FB
n_hat_force_compensation_source = plant.simulation.NHatForceCompensationSource.CONSTANT

In [ ]:
sim = plant.simulation.Simulation(
    ctrl_paradigm=ctrl_paradigm,
    impedance_type=impedance_type,
    n_hat_force_compensation_source=n_hat_force_compensation_source,
    meshcat=meshcat
)

### Final diagram

Print SVG:

In [ ]:
#SVG(pydot.graph_from_dot_data(sim.get_viz_str())[0].create_svg())

Print PNG:

In [ ]:
Image(pydot.graph_from_dot_data(sim.get_viz_str())[0].create_png())

# Run simulation

In [ ]:
log = sim.run_sim()

In [ ]:
# This way, we can use the actual time the simulation ran for
effective_tspan = max(log.sample_times())
print("effective_tspan:", effective_tspan)

# Define helper functions

In [ ]:
# Friction helper functions
def step5(x):
    '''Python version of MultibodyPlant::StribeckModel::step5 method'''
    x3 = x * x * x
    return x3 * (10 + x * (6 * x - 15))

def stribeck(us, uk, v):
    '''
    Python version of MultibodyPlant::StribeckModel::ComputeFrictionCoefficient
    
    From
    https://github.com/RobotLocomotion/drake/blob/b09e40db4b1c01232b22f7705fb98aa99ef91f87/multibody/plant/images/stiction.py
    '''
    u = np.zeros_like(v) + uk
    u[v < 1] = us * step5(v[v < 1])
    mask = (v >= 1) & (v < 3)
    u[mask] = us - (us - uk) * step5((v[mask] - 1) / 2)
    return u

In [ ]:
def T(vec):
    return np.transpose(vec, [0, 2, 1])

## `diff` functions

In [ ]:
def diff_vec(vec):
    d_vec = np.diff(vec, axis=0)
    for i in range(vec.shape[1]):
        d_vec[:, i] /= np.expand_dims(np.diff(l_times), 1)
    d_vec = np.concatenate((d_vec, [d_vec[-1]]))
    return d_vec

In [ ]:
def diff_arr(arr):
    d_arr = np.diff(arr)/np.diff(l_times)
    d_arr = np.concatenate((d_arr, [d_arr[-1]]))
    return d_arr

## `plot` functions

In [ ]:
def plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True, start_time=None):
    if start_time is None:
        idx_start_ = idx_start
    else:
        idx_start_ = np.argmax(l_times > start_time)
    plt.figure(figsize=(16, 8))
    lw = 5
    if plot_x:
        plt.plot(l_times[idx_start_:idx_end], sig1[:,0][idx_start_:idx_end], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], sig1[:,1][idx_start_:idx_end], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], sig1[:,2][idx_start_:idx_end], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)
    
    if plot_x:
        plt.plot(l_times[idx_start_:idx_end], sig2[:,0][idx_start_:idx_end], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], sig2[:,1][idx_start_:idx_end], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], sig2[:,2][idx_start_:idx_end], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def plot_vec_HTN(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True, start_time=None):
    if start_time is None:
        idx_start_ = idx_start
    else:
        idx_start_ = np.argmax(l_times > start_time)
    plt.figure(figsize=(16, 8))
    lw = 5
    
    T_proj1 = np.matmul(T(sig1), T_hat).flatten()
    T_proj2 = np.matmul(T(sig2), T_hat).flatten()
    N_proj1 = np.matmul(T(sig1), N_hat).flatten()
    N_proj2 = np.matmul(T(sig2), N_hat).flatten()
    
    plt.plot(l_times[idx_start_:idx_end], sig1[:,hinge_rotation_axis][idx_start_:idx_end], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], T_proj1[idx_start_:idx_end], label=sig1_label + r' $\hat T$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], N_proj1[idx_start_:idx_end], label=sig1_label + r' $\hat N$ component', color='lightskyblue', linewidth=lw)
    
    plt.plot(l_times[idx_start_:idx_end], sig2[:,hinge_rotation_axis][idx_start_:idx_end], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], T_proj2[idx_start_:idx_end], label=sig2_label + r' $\hat T$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], N_proj2[idx_start_:idx_end], label=sig2_label + r' $\hat N$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def plot_eq(lhs, lhs_label, rhs, rhs_label, lhs_alpha=1, rhs_alpha=1, lw=5, finish_plotting=True, 
            plot_diff=True, figsize_=None, plot_err_as_pct=False, t_start_=None, newline_in_title=False,
            t_end_=None):
    # Can't set these to be default params because I don't want to have to define t_start
    # before this function is defined
    if t_start_ is None:
        t_start_ = t_start
    if t_end_ is None:
        t_end_ = t_end

    if figsize_ is not None:
        plt.figure(figsize=figsize_)
    else:
        plt.figure(figsize=default_figsize)
    if isinstance(lhs, float) or isinstance(lhs, int):
        lhs = np.ones_like(l_times)*lhs
    if isinstance(rhs, float) or isinstance(rhs, int):
        rhs = np.ones_like(l_times)*rhs
    idx_start = np.argmax(l_times > t_start_)
    idx_end = np.argmax(l_times > t_end_)
    if idx_end == 0:
        idx_end = len(l_times) - 1
        
    b = np.broadcast(lhs, rhs)
    if np.count_nonzero(np.array(b.shape) > 1000) > 1:
        raise ValueError("Broadcast shape of lhs and rhs is {}, which is too large.".format(b.shape))
        
    plt.subplot(211)
    error = lhs[idx_start:idx_end]-rhs[idx_start:idx_end]
    if plot_err_as_pct:
        lhs_range = np.nanmax(lhs[idx_start:idx_end]) - np.nanmin(lhs[idx_start:idx_end])
        rhs_range = np.nanmax(rhs[idx_start:idx_end]) - np.nanmin(rhs[idx_start:idx_end])
        
        lhs_max_val = np.nanmax(np.abs(lhs[idx_start:idx_end]))
        rhs_max_val = np.nanmax(np.abs(rhs[idx_start:idx_end]))
        error /= np.nanmax([lhs_max_val, rhs_max_val])
        error *= 100
        
    plt.plot(l_times[idx_start:idx_end],
            error,
            linewidth=lw)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlabel("Time (s)")
    if plot_err_as_pct:
        plt.ylabel("Error (percent of max val)")
    else:
        plt.ylabel("Error")
    if newline_in_title:
        plt.title("Difference between {}\n and {}".format(lhs_label, rhs_label))
    else:
        plt.title("Difference between {} and {}".format(lhs_label, rhs_label))
    
    
    plt.subplot(212)
    plt.plot(l_times[idx_start:idx_end],
             lhs[idx_start:idx_end],
             label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    plt.plot(l_times[idx_start:idx_end],
             rhs[idx_start:idx_end],
             label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    plt.autoscale(enable=True, axis='x', tight=True)
    
#     plt.axvline(t[np.argmax(in_contact)] + 0.5, color='k', linewidth=lw, linestyle='--')
    
    
#     plt.axvline(t[first_contact_idx], color='green', linewidth=lw)
#     plt.axvline(t_lose_contact, color='red', linewidth=lw)
    
    plt.legend()
    plt.xlabel("Time (s)")
    
    if finish_plotting:
        plt.show()

# Load signals

## Load times and `debug` dict

In [ ]:
if hasattr(sim.fold_ctrl, "debug"):
    debug = sim.fold_ctrl.debug
    
    min_key_length = np.inf
    for k in debug.keys():
        if len(debug[k]) > 0 and len(debug[k]) < min_key_length:
            print(k)
            min_key_length = len(debug[k])
    for k in debug.keys():
        debug[k] = debug[k][:min_key_length]

    d_times = np.array(debug['times'])
else:
    debug = None

In [ ]:
l_times = log.sample_times()

## Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.

In [ ]:
nq = manipulator.data['nq']

### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = sim.sys_consts.m_L
w_L = sim.sys_consts.m_L
I_L = sim.sys_consts.I_L
h_L = sim.sys_consts.h_L
# mu = 2*mu_paper/(1+mu_paper) # 2μₘμₙ/(μₘ + μₙ)
mu = sim.sys_consts.mu
b_J = sim.sys_consts.b_J
k_J = sim.sys_consts.k_J

### Gravity, unit vectors ($F_{GT}, F_{GN}, \hat T, \hat N$)

In [ ]:
# Assume plane contact, so all N_hat should point the same direction
# TODO: verify all point in the same direction
N_hat_raw = log.data()[sim.log_wrapper.contact_entry_start_idx+14:sim.log_wrapper.contact_entry_start_idx+17,:].copy() # So it's writeable
N_hat = np.expand_dims(N_hat_raw.T, 2)
N_hat[:,0][np.isnan(N_hat[:,0])] = 0
N_hat[:,1][np.isnan(N_hat[:,1])] = 0
N_hat[:,2][np.isnan(N_hat[:,2])] = 1

In [ ]:
H_hat = np.zeros_like(N_hat)
H_hat[:,hinge_rotation_axis] = 1

In [ ]:
# T hat can be defined by rotating N hat about hinge axis by -90 degrees
rot = RotationMatrix(AngleAxis(axis=H_hat[0].flatten(), angle=-np.pi/2)).matrix()

T_hat = np.matmul(rot, N_hat)

It's also useful here to calculate project matrices:

In [ ]:
N_hat_proj_mat = np.matmul(N_hat, np.transpose(N_hat, [0, 2, 1]))

In [ ]:
g = sim.plant.gravity_field().gravity_vector()[-1]*-1
F_G = np.zeros([l_times.size, 3, 1])
F_G[:,2,0] = -np.ones_like(l_times)*g*m_L
F_GT = np.matmul(T(F_G), T_hat).flatten()
F_GN = np.matmul(T(F_G), N_hat).flatten()

In [ ]:
idx = sim.log_wrapper.tau_g_start_idx
length = manipulator.data["nq"]
tau_g = np.expand_dims(log.data()[idx:idx+length].T, 2)

### Positions, velocities ($\dot\theta^2, r_T, \dot d_T$)

#### Object poses & vels

In [ ]:
idx = sim.log_wrapper.get_idx("pos", "trn", sim.ll_idx)
length = 3
p_L = np.expand_dims(log.data()[idx:idx+length].T, 2)
p_LN = np.matmul(T(p_L), N_hat).flatten()
p_LT = np.matmul(T(p_L), T_hat).flatten()
p_LH = p_L[:,hinge_rotation_axis].flatten()

In [ ]:
theta_LX = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx)]
theta_LY = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx)+1]
theta_LZ = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx)+2]

In [ ]:
theta_L = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx) + hinge_rotation_axis].copy()
# Fix issue in RPY singularity
theta_L[theta_LZ > np.pi/2] = theta_L[theta_LZ > np.pi/2]*-1 + np.pi

In [ ]:
theta_MX = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.contact_body_idx)]
theta_MY = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.contact_body_idx)+1]
theta_MZ = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.contact_body_idx)+2]

In [ ]:
d_theta_L = log.data()[sim.log_wrapper.get_idx("vel", "rot", sim.ll_idx) + hinge_rotation_axis]
d_theta_M = log.data()[sim.log_wrapper.get_idx("vel", "rot", sim.contact_body_idx)]

In [ ]:
x_hat = np.zeros((len(l_times), 3, 1))
x_hat[:,0,:] = 1

y_hat = np.zeros((len(l_times), 3, 1))
y_hat[:,1,:] = 1

z_hat = np.zeros((len(l_times), 3, 1))
z_hat[:,2,:] = 1

In [ ]:
idx = sim.log_wrapper.get_idx("vel", "trn", sim.ll_idx)
length = 3
v_L = np.expand_dims(log.data()[idx:idx+length].T,2)
v_LN = np.matmul(T(v_L), N_hat).flatten()
v_LT = np.matmul(T(v_L), T_hat).flatten()
v_LH = v_L[:,hinge_rotation_axis].flatten()

In [ ]:
idx = sim.log_wrapper.get_idx("pos", "trn", sim.contact_body_idx)
length = 3
p_M = np.expand_dims(log.data()[idx:idx+length].T, 2)
p_MN = np.matmul(T(p_M), N_hat).flatten()
p_MT = np.matmul(T(p_M), T_hat).flatten()
p_MH = p_M[:,hinge_rotation_axis].flatten()

In [ ]:
idx = sim.log_wrapper.get_idx("pos", "rot", sim.contact_body_idx)
length = 3
np.expand_dims(log.data()[idx:idx+length].T, 2)[0]

In [ ]:
idx = sim.log_wrapper.get_idx("vel", "trn", sim.contact_body_idx)
length = 3
v_M = np.expand_dims(log.data()[idx:idx+length].T, 2)
v_MN = np.matmul(T(v_M), N_hat).flatten()
v_MT = np.matmul(T(v_M), T_hat).flatten()
v_MH = v_M[:,hinge_rotation_axis].flatten()

#### Manipulator eq. terms

In [ ]:
idx = sim.log_wrapper.J_start_idx
length = manipulator.data["nq"]*6
J = np.transpose(log.data()[idx:idx+length].reshape(6, nq, len(l_times)), [2, 0, 1])
J_rotational = J[:,:3,:]
J_translational = J[:,3:,:]

In [ ]:
idx = sim.log_wrapper.joint_centering_torque_start_idx
length = manipulator.data["nq"]
joint_centering_torque = np.expand_dims(np.transpose(log.data()[idx:idx+length]), 2)

In [ ]:
q = np.expand_dims((log.data()[sim.log_wrapper.state_start_idx:sim.log_wrapper.state_start_idx + nq,:]).T, 2)
d_q = np.expand_dims((log.data()[sim.log_wrapper.state_start_idx + nq:sim.log_wrapper.state_start_idx + nq*2,:]).T, 2)

In [ ]:
idx = sim.log_wrapper.M_start_idx
length = manipulator.data["nq"]*manipulator.data["nq"]
M = np.transpose(log.data()[idx:idx+length].reshape(nq, nq, len(l_times)), [2, 0, 1])

In [ ]:
idx = sim.log_wrapper.Cv_start_idx
length = manipulator.data["nq"]
Cv = np.expand_dims(log.data()[idx:idx+length].T, 2)

In [ ]:
idx = sim.log_wrapper.Jdot_qdot_start_idx
length = 6
Jdot_qdot = np.expand_dims(log.data()[idx:idx+length].T, 2)

#### Body derived terms
Meaning terms derived from the positions/velocities of bodies

In [ ]:
R = np.zeros((len(l_times), 3, 3))
R[:,1,1] = np.cos(theta_L)
R[:,1,2] = -np.sin(theta_L)
R[:,2,1] = np.sin(theta_L)
R[:,2,2] = np.cos(theta_L)

In [ ]:
T_hat_geo = np.matmul(R, y_hat)
N_hat_geo = np.matmul(R, z_hat)

In [ ]:
# Link edge
p_LE = p_L + (w_L/2)*T_hat-(h_L/2)*N_hat

In [ ]:
omega_vec_L = np.expand_dims(np.vstack((d_theta_L, np.zeros_like(d_theta_L), np.zeros_like(d_theta_L))).T, 2)
idx = sim.log_wrapper.get_idx("vel", "rot", sim.contact_body_idx)
length = 3
omega_vec_M = np.expand_dims(log.data()[idx:idx+length].T, 2)

In [ ]:
J_dot = diff_vec(J)

#### Contact terms
TODO: edit out/simplify eventually

In [ ]:
contact_point = log.data()[sim.log_wrapper.contact_entry_start_idx+11:sim.log_wrapper.contact_entry_start_idx+14,:]

In [ ]:
p_C = np.expand_dims(contact_point.T, 2)
p_CT = np.matmul(T(p_C), T_hat).flatten()
p_CN = np.matmul(T(p_C), N_hat).flatten()

In [ ]:
r = np.linalg.norm(p_C - p_M, axis=1).flatten()

In [ ]:
# Assume all the slip speeds are the same
# TODO: verify
slip_speed = log.data()[sim.log_wrapper.contact_entry_start_idx+10,:]

In [ ]:
# d vector
d_vec = p_C - p_LE
d_T = np.matmul(T(d_vec), T_hat).flatten()
d_N = np.matmul(T(d_vec), N_hat).flatten()
d_H = d_vec[:,hinge_rotation_axis].flatten()

In [ ]:
d_d_T = -d_theta_L*h_L/2-d_theta_L*r - v_LT + v_MT + d_theta_L*d_N
d_d_N = -d_theta_L*w_L/2-v_LN+v_MN-d_theta_L*d_T
d_d_H = diff_arr(d_H)

In [ ]:
dd_d_T = diff_arr(d_d_T)
dd_d_N = diff_arr(d_d_N)
dd_d_H = diff_arr(d_d_H)

In [ ]:
d_d_vec = diff_vec(d_vec)
d_d_vec_T = np.matmul(T(d_d_vec), T_hat).flatten()
d_d_vec_N = np.matmul(T(d_d_vec), N_hat).flatten()

In [ ]:
p_MConM = p_C - p_M
p_LConL = p_C - p_L

v_WConM = v_M + np.cross(omega_vec_M, p_MConM, axis=1)
v_WConL = v_L + np.cross(omega_vec_L, p_LConL, axis=1)

v_S_raw = v_WConM - v_WConL
v_S_N = np.matmul(N_hat_proj_mat, v_S_raw)
v_S = v_S_raw - v_S_N
v_S = v_S_raw - v_S_N
s_S = np.linalg.norm(v_S, axis=1).flatten()

In [ ]:
s_hat = v_S/np.expand_dims(np.linalg.norm(v_S, axis=1), 2)
s_hat_H = s_hat[:,hinge_rotation_axis].flatten()
s_hat_T = np.matmul(T(s_hat), T_hat).flatten()

In [ ]:
d_d_T_diff = diff_arr(d_T)
d_d_N_diff = diff_arr(d_N)

### Friction coefficients

In [ ]:
mu_S = stribeck(1, 1, slip_speed/constants.v_stiction)

## Outputs
These should all come directly from simulation, except for control forces.
### Measured accelerations ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
idx = sim.log_wrapper.get_idx("acc", "trn", sim.ll_idx)
length = 3
a_L = np.expand_dims(log.data()[idx:idx+length].T, 2)
a_LH = np.matmul(T(a_L), H_hat).flatten()
a_LT = np.matmul(T(a_L), T_hat).flatten()
a_LN = np.matmul(T(a_L), N_hat).flatten()

In [ ]:
idx = sim.log_wrapper.get_idx("acc", "trn", sim.contact_body_idx)
length = 3
a_M = np.expand_dims(log.data()[idx:idx+length].T, 2)
a_MH = np.matmul(T(a_L), H_hat).flatten()
a_MT = np.matmul(T(a_M), T_hat).flatten()
a_MN = np.matmul(T(a_M), N_hat).flatten()
a_MX = a_M[:,0].flatten()
a_MY = a_M[:,1].flatten()
a_MZ = a_M[:,2].flatten()

In [ ]:
idx = sim.log_wrapper.get_idx("acc", "rot", sim.contact_body_idx)
length = 3
alpha_M = np.expand_dims(log.data()[idx:idx+length].T, 2)
alpha_MT = np.matmul(T(alpha_M), T_hat).flatten()
alpha_MN = np.matmul(T(alpha_M), N_hat).flatten()
alpha_MX = alpha_M[:,0].flatten()
alpha_MY = alpha_M[:,1].flatten()
alpha_MZ = alpha_M[:,2].flatten()

In [ ]:
dd_theta_L = log.data()[sim.log_wrapper.get_idx("acc", "rot", sim.ll_idx) + hinge_rotation_axis]
dd_theta_M = log.data()[sim.log_wrapper.get_idx("acc", "rot", sim.contact_body_idx)]

In [ ]:
dd_q = np.expand_dims((log.data()[sim.log_wrapper.gen_accs_start_idx:
                                     sim.log_wrapper.gen_accs_start_idx+nq,:]).T, 2)

### Contact forces ($F_{NL}, F_{FL}, F_{NM}, F_{FM}$)
Slightly different then just taking normal/tangent projections.

In [ ]:
F_NM = np.zeros(len(l_times))
F_FMT = np.zeros(len(l_times))
F_FMH = np.zeros(len(l_times))
F_ContactM_XYZ = np.zeros((len(l_times), 3, 1))
tau_contact = np.zeros((len(l_times), nq, 1))
for i in range(sim.log_wrapper.max_contacts):
    # Pull out terms
    start_idx = sim.log_wrapper.contact_entry_start_idx + sim.log_wrapper.entries_per_contact*i
    contact_point = np.expand_dims(log.data()[start_idx+11:start_idx+14,:].T, 2)
    if np.all(np.isnan(contact_point)):
        break

    # Calculate intermediautes
    lever_arm = contact_point - p_M
    force = np.expand_dims(log.data()[start_idx+0:start_idx+3,:].T, 2)
    torque = np.cross(lever_arm, force, axis=1)
    wrench = np.hstack((torque, force))
    
    # Calculate updates
    F_FMT_update = np.matmul(T(force), T_hat).flatten()
    F_NM_update = np.matmul(T(force), N_hat).flatten()
    F_FMH_update = force[:,hinge_rotation_axis].flatten()
    F_ContactM_XYZ_update = force
    
    # Update
    tau_contact += np.matmul(T(J), wrench)
    F_FMT += F_FMT_update
    F_NM += F_NM_update
    F_FMH += F_FMH_update
    F_ContactM_XYZ += F_ContactM_XYZ_update
    
F_NM = np.nan_to_num(F_NM)
F_FMT = np.nan_to_num(F_FMT)
F_FMH = np.nan_to_num(F_FMH)
tau_contact = np.nan_to_num(tau_contact)
F_ContactM_XYZ = np.nan_to_num(F_ContactM_XYZ)

In [ ]:
F_FLT = -F_FMT
F_FLH = -F_FMH
F_NL = -F_NM

In [ ]:
# Equivalent contact forces for each object
F_contact_M = np.expand_dims(F_FMH, [1, 2])*H_hat + \
                  np.expand_dims(F_FMT, [1, 2])*T_hat + \
                  np.expand_dims(F_NM, [1, 2])*N_hat

In [ ]:
F_ContactMY = F_contact_M[:,1,0].flatten()
F_ContactMZ = F_contact_M[:,2,0].flatten()

In [ ]:
F_contact_L = np.expand_dims(F_FLH, [1, 2])*H_hat + \
              np.expand_dims(F_FLT, [1, 2])*T_hat + \
              np.expand_dims(F_NL, [1, 2])*N_hat

### Object forces ($F_{ON}, F_{OT}, \tau_O$) and other joint terms

In [ ]:
idx = sim.log_wrapper.get_last_jnt_idx("trn")
joint_force_in_compliance_frame = log.data()[idx:idx+3]

In [ ]:
# Copy makes it mutable
# TODO: figure out how to make this more reliable
F_OH = joint_force_in_compliance_frame[1,:].copy()
F_OT = -joint_force_in_compliance_frame[0,:].copy() # X direction is opposite of T hat
F_ON = joint_force_in_compliance_frame[2,:].copy()

In [ ]:
idx = sim.log_wrapper.get_last_jnt_idx("rot")
joint_torque_in_compliance_frame = log.data()[idx:idx+3]

In [ ]:
# Copy makes it mutable
tau_O = joint_torque_in_compliance_frame[hinge_rotation_axis,:].copy()

In [ ]:
F_OH = np.nan_to_num(F_OH)
F_OT = np.nan_to_num(F_OT)
F_ON = np.nan_to_num(F_ON)
tau_O = np.nan_to_num(tau_O)

In [ ]:
# Get transform from link CoM to joint EXPRESSED IN LINK FRAME
X_LJ_L = sim.paper.joints[0].frame_on_child().GetFixedPoseInBodyFrame()

X_WJ = []
p_JL = []
p_JC = []
for p_L_, p_C_, theta_LX_, theta_LY_, theta_LZ_ in zip(p_L, p_C, theta_LX, theta_LY, theta_LZ):
    # Construct transform of link CoM
    X_WL_ = RigidTransform(
        p=p_L_,
        rpy=RollPitchYaw(theta_LX_, theta_LY_, theta_LZ_)
    )
    
    # Get joint location EXPRESSED IN WORLD FRAME
    X_WJ = X_WL_.multiply(X_LJ_L)
    
    # Calculate lever arms for later torque math
    p_JL.append(p_L_.flatten() - X_WJ.translation())
    p_JC.append(p_C_.flatten() - X_WJ.translation())
p_JL = np.expand_dims(np.array(p_JL), 2)
p_JC = np.expand_dims(np.array(p_JC), 2)

In [ ]:
# Get inertia relative to joint of link, via parallel axis theorem
d_ = np.linalg.norm(p_JL, axis=1).flatten()
I_LJ = I_L + m_L*d_**2

## Control terms

In [ ]:
idx = sim.log_wrapper.tau_ctrl_start_idx
length = manipulator.data["nq"]
tau_ctrl = np.expand_dims(np.transpose(log.data()[idx:idx+length]), 2)

In [ ]:
idx = sim.log_wrapper.tau_out_start_idx
length = manipulator.data["nq"]
tau_out = np.expand_dims(np.transpose(log.data()[idx:idx+length]), 2)

In [ ]:
F_out = np.matmul(J_translational, tau_out)
F_outN = np.matmul(T(F_out), N_hat).flatten()
F_outT = np.matmul(T(F_out), T_hat).flatten()

### Other outputs

In [ ]:
forces_found = log.data()[-1,:]

In [ ]:
in_contact = log.data()[sim.log_wrapper.calc_in_contact_start_idx]

In [ ]:
t = l_times

# Set up plotting times

## Process contact times

These are all derived by contact, so we should first inspect the contact plot:

In [ ]:
plt.plot(l_times, in_contact, "b--", linewidth=2, label="Filtered")
plt.legend()
plt.show()

Select what data should be use to select contact:

In [ ]:
contact_data = in_contact

In [ ]:
first_contact_idx = np.argmax(contact_data)
last_contact_idx = contact_data.size - np.argmax(contact_data[::-1]-1)-1
l_times[last_contact_idx]

In [ ]:
last_not_in_contact_idx = contact_data.size - np.argmax(np.logical_not(contact_data[::-1])-1)-1
t_start_of_last_contact = t[min(last_not_in_contact_idx+1, len(l_times) - 1)]

## Select final `t_start` and `t_end`

In [ ]:
# t_start = 0
t_start = t[first_contact_idx]

In [ ]:
t_end = effective_tspan
# t_end = t[last_contact_idx]-0.5

In [ ]:
idx_start = np.argmax(l_times > t_start)
idx_end = np.argmax(l_times > t_end)
if idx_end == 0:
    idx_end = len(l_times) - 1

In [ ]:
t_start

In [ ]:
t_end

# Plots (individual run)

## Environmental constraints
There are things where I expect them to be satisfied no matter what, no matter the success of my controller.

### Set up

In [ ]:
errors = []
max_errors = []
med_errors = []
max_error_pct = []
med_error_pct = []
error_labels = []

In [ ]:
def update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=False):
    error = ((lhs) - (rhs))[idx_start:idx_end]
    
    if isinstance(lhs, float) or isinstance(lhs, int):
        lhs = np.ones_like(l_times)*lhs
    if isinstance(rhs, float) or isinstance(rhs, int):
        rhs = np.ones_like(l_times)*rhs
    max_val_l = np.max(np.abs(lhs[idx_start:idx_end]))
    max_val_r = np.max(np.abs(rhs[idx_start:idx_end]))
    max_val = max(max_val_l,max_val_r)
    
    label = r"$\left(" + lhs_label.replace("$", "") + r"\right) "
    if newline_in_label:
        label += "$\n$"
    label += "- \left(" + rhs_label.replace("$", "") + r"\right)$"
    label_idx = None
    for i, l in enumerate(error_labels):
        if l == label:
            label_idx = i
            break
    max_error = np.nanmax(np.abs(error))
    med_error = np.median(np.abs(error[np.logical_not(np.isnan(error))]))
    if label_idx is None:
        errors.append(error)
        error_labels.append(label)
        max_errors.append(max_error)
        med_errors.append(med_error)
        max_error_pct.append(max_error/max_val)
        med_error_pct.append(max_error/max_val)
    else:
        errors[label_idx] = error
        max_errors[label_idx] = max_error
        med_errors[label_idx] = med_error
        max_error_pct[label_idx] = max_error/max_val
        med_error_pct[label_idx] = med_error/max_val

### Plots

#### Link (H) accel $m_La_{LH} = F_{FLH} + F_{OX}$

In [ ]:
lhs_label = r'$m_La_{LH}$'
rhs_label = r'$F_{FLH} + F_{OH}$'

lhs = m_L*a_LH

rhs = F_FLH + F_OH

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Link (T) accel $m_La_{LT} = F_{FLT} + F_{GT} +F_{OT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FLT} + F_{GT} +F_{OT}$'

lhs = m_L*a_LT
# lhs /= np.max(np.abs(lhs[idx_start:idx_end]))

rhs = F_GT+F_OT+F_FLT
# rhs /= np.max(np.abs(rhs[idx_start:idx_end]))

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Link (N) accel $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat N$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

lhs = m_L*a_LN

rhs = F_NL + F_GN + F_ON

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Link (H) angle accel $I_{LJ}\ddot\theta_L = p_{JC}\times F_{contact} + p_{JL}\times F_G + \tau_O$
*If this plot is correct:* The moment balance about the joint on the link is correct.

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$p_{JC}\times F_{contact} + p_{JL}\times F_G + \tau_O$'

lhs = I_LJ*dd_theta_L

contact_torque_about_joint = np.cross(p_JC, F_contact_L, axis=1)[:,hinge_rotation_axis].flatten()
gravity_torque_about_joint = np.cross(p_JL, F_G, axis=1)[:,hinge_rotation_axis].flatten()
rhs = gravity_torque_about_joint + contact_torque_about_joint + tau_O

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### 3rd law normal forces $F_{NL} = -F_{NM}$
*If this plot is correct:* Newton's 3rd law is correction applied to the normal forces.

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM_{top}}-F_{NM_{bot}}$'

lhs = F_NL

rhs = -F_NM

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### 3rd law friction forces $F_{FLT} = -F_{FMT}$
*If this plot is correct:* Newton's 3rd law is correction applied to the friction forces in the tangent direction.

In [ ]:
lhs_label = r'$F_{FLT}$'
rhs_label = r'$-F_{FMT}$'

lhs = F_FLT

rhs = -F_FMT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### d vec tangential derivative $-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT} = -\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*(h_L/2+r) + d_theta_L**2*w_L/2 - a_LT + a_MT
lhs_label = r"$-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT}$"

rhs = -dd_theta_L*d_N + dd_d_T - d_theta_L**2*d_T - 2*d_theta_L*d_d_N
rhs_label = r"$-\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$"

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)

#### d vec normal derivative $- \frac{w_{L}}{2}\ddot\theta_{L} - \frac{h_{L}}{2}\dot\theta_{L}^{2} - \dot\theta_{L}^{2} r - a_{LN} + a_{MN} = \ddot\theta_{L} d_{T} + \ddot{d}_N - \dot\theta_{L}^{2} d_{N} + 2 \dot\theta_{L} \dot{d}_T$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*w_L/2 - d_theta_L**2*h_L/2 - d_theta_L**2*r - a_LN + a_MN
lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$ (differentiating geometry)"

rhs = dd_theta_L*d_T + dd_d_N - d_theta_L**2*d_N + 2*d_theta_L*d_d_T
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$ (differentiating vector)"

plot_eq(lhs, lhs_label, rhs, rhs_label, newline_in_title=True)

lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$"
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$"
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)
plt.show()

#### No penetration $\ddot d_N = 0$
*If this plot is correct:* There is effectively no penetration between the rigid bodies.

In [ ]:
lhs_label = r'$\ddot{d}_N$'
rhs_label = r'0'

lhs = dd_d_N

rhs = 0

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Link (T) friction equation $F_{FLT}=\mu\mu_SF_{NL}\hat s_T$

In [ ]:
lhs_label = r'$\mu\mu_SF_{NL}\hat s_T$'
rhs_label = r'$F_{FLT}$'

lhs = np.nan_to_num(mu_S*F_NL*mu*s_hat_T)
rhs = F_FLT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Link (x) friction equation $F_{FLX}=\mu\mu_SF_{NL}\hat s_X$

In [ ]:
lhs_label = r'$\mu\mu_SF_{NL}\hat s_H$'
rhs_label = r'$F_{FLH}$'

lhs = np.nan_to_num(mu_S*F_NM*mu*s_hat_H)
rhs = F_FMH

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (x) accel $a_{MX}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_3$

In [ ]:
lhs_label = r'$a_{MX}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_3$'

lhs = a_MX
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,3].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (y) accel $a_{MY}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_4$

In [ ]:
lhs_label = r'$a_{MY}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_4$'

lhs = a_MY
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,4].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (z) accel $a_{MZ}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_5$

In [ ]:
lhs_label = r'$a_{MZ}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_5$'

lhs = a_MZ
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,5].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (x) ang accel $\alpha_{MX}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_0$

In [ ]:
lhs_label = r'$\alpha_{MX}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_0$'

lhs = alpha_MX
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,0].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (y) ang accel $\alpha_{MY}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_1$

In [ ]:
lhs_label = r'$\alpha_{MY}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_1$'

lhs = alpha_MY
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,1].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Jacobian and (z) ang accel $\alpha_{MZ}=\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_2$

In [ ]:
lhs_label = r'$\alpha_{MZ}$'
rhs_label = r'$\left(\dot{J} \dot{\vec{q}} + J\ddot{\vec{q}}\right)_2$'

lhs = alpha_MZ
rhs = (Jdot_qdot + np.matmul(J, dd_q))[:,2].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

#### Manipulator equation $M \ddot q + C \dot q = \tau_g + \vec \tau_{\text{contact}} + \vec \tau_{\text{out}}$

In [ ]:
lhs_ = np.matmul(M, dd_q) + Cv
lhs_label_ = r"M \ddot q + C \dot q"
rhs_ = tau_g + tau_contact + tau_out
rhs_label_ = r"\tau_g + \vec \tau_{contact} + \vec \tau_{out}"
labels = [str(i) for i in range(nq)]

for i, label in enumerate(labels):
    lhs = lhs_[:,i,:]
    lhs_label = r"$\left(" + lhs_label_ + r"\right)_" + label + "$"

    rhs = rhs_[:,i,:]
    rhs_label = r"$\left(" + rhs_label_ + r"\right)_" + label + "$"

    plot_eq(lhs, lhs_label, rhs, rhs_label)
    update_error(lhs, lhs_label, rhs, rhs_label)
    plt.show()

#### Tau out vs tau ctrl $\vec \tau_{\text{out}} = \vec\tau_{\text{ctrl}} - \tau_g + \tau_{\text{joint centering}}$

In [ ]:
lhs_ = tau_out
lhs_label_ = r"\tau_{out}"
rhs_ = tau_ctrl - tau_g + joint_centering_torque
rhs_label_ = r"\vec\tau_{ctrl} - \tau_g + \tau_{joint centering}"
labels = [str(i) for i in range(nq)]

for i, label in enumerate(labels):
    lhs = lhs_[:,i,:]
    lhs_label = r"$\left(" + lhs_label_ + r"\right)_" + label + "$"

    rhs = rhs_[:,i,:]
    rhs_label = r"$\left(" + rhs_label_ + r"\right)_" + label + "$"

    plot_eq(lhs, lhs_label, rhs, rhs_label)
    update_error(lhs, lhs_label, rhs, rhs_label)
    plt.show()

### Plot errors
How accurate is my modeling?

#### Absolute error

In [ ]:
plt.figure(figsize=(16,16))
plt.barh(error_labels[::-1], np.array(max_errors[::-1]), label="Maximum error")
plt.barh(error_labels[::-1], np.array(med_errors[::-1]), label="Median error")
plt.axvline(1e-5, color='k', linestyle='--', label="1e-5")
plt.xscale("log")
plt.xlabel("Log(error)")
plt.xlabel("Error")
plt.ylabel("Equation")
plt.legend()
# plt.xlim(0, 100)
plt.show()

#### Percent error

In [ ]:
plt.figure(figsize=(16,16))
plt.barh(error_labels[::-1], np.array(max_error_pct[::-1])*100, label="Maximum error")
plt.barh(error_labels[::-1], np.array(med_error_pct[::-1])*100, label="Median error")
plt.axvline(5, color='k', linestyle='--', label="5% error")
# plt.xscale("log")
# plt.xlabel("Log(error)")
plt.xlabel("Percent error")
plt.ylabel("Equation")
plt.legend()
plt.xlim(0, 100)
plt.show()

## Environmental constraints (two link specific)

In [ ]:
lhs = a_LT
lhs_label = r"$a_{LT}$"
rhs = (-w_L/2)*d_theta_L**2
rhs_label = r"$-\frac{w_L}{2}\dot\theta^2$"
plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
lhs = a_LN
lhs_label = r"$a_{LN}$"
rhs = (w_L/2)*dd_theta_L
rhs_label = r"$\frac{w_L}{2}\ddot\theta$"
plot_eq(lhs, lhs_label, rhs, rhs_label)

## Controller performance

### Convergence to desired values

#### $\theta_{MX}$

In [ ]:
if not (debug is None) and "theta_MXd" in debug.keys():
    theta_MXd = scipy.interpolate.interp1d(d_times, debug['theta_MXd'], axis=0)(l_times)
    plot_eq(theta_MX, r"$\theta_{MX}$", theta_MXd, r"$\theta_{MXd}$")

#### $\theta_{MY}$

In [ ]:
if hasattr(sim.fold_ctrl, "theta_MYd"):
    plot_eq(theta_MY, r"$\theta_{MY}$", sim.fold_ctrl.theta_MYd, r"$\theta_{MYd}$")

#### $\theta_{MZ}$

In [ ]:
if hasattr(sim.fold_ctrl, "theta_MZd"):
    plot_eq(theta_MZ, r"$\theta_{MZ}$", sim.fold_ctrl.theta_MZd, r"$\theta_{MZd}$")

#### $\theta_{L}$

In [ ]:
if hasattr(sim.fold_ctrl, "d_theta_Ld"):
    plot_eq(d_theta_L, r"$\dot\theta_{L}$", sim.fold_ctrl.d_theta_Ld, r"$\dot\theta_{Ld}$")

### Inverse dynamics specific

#### $d_{T_d}$

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    plot_eq(d_T, "$d_T$", sim.fold_ctrl.d_Td, "$d_{Td}$")

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lw=5
    plt.figure(figsize=(16, 8))
    plt.axhline(sim.fold_ctrl.d_Td, linestyle='--', color='gray', linewidth=lw, label="Desired $d_T$")
    plt.plot(l_times, d_T, label='$d_T$', linewidth=lw)
    plt.axhline(-(0), label='Bounds on $d_T$ (0 and link width)', linestyle='--', color='k', linewidth=lw)
    plt.axhline(-(paper.link_width), linestyle='--', color='k', linewidth=lw)
    plt.legend()
    plt.xlabel("Time (seconds)")
    plt.ylabel("Distance (m)")
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlim(t_start, t_end)
    plt.show()

#### Conformity to artificial constraints

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = dd_d_Td
    lhs_label = r"$\ddot{d}_{T_d}$"
    rhs = dd_d_T
    rhs_label = r"$\ddot{d}_{T}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = dd_theta_Ld
    lhs_label = r"$a_{LN_d}$"
    rhs = dd_theta_L
    rhs_label = r"$a_{LN}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = a_MXd
    lhs_label = r"$a_{MX_d}$"
    rhs = a_MX
    rhs_label = r"$a_{MX}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = alpha_MXd
    lhs_label = r"$\alpha_{MX_d}$"
    rhs = alpha_MX
    rhs_label = r"$\alpha_{MX}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = alpha_MYd
    lhs_label = r"$\alpha_{MY_d}$"
    rhs = alpha_MY
    rhs_label = r"$\alpha_{MY}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = alpha_MZd
    lhs_label = r"$\alpha_{MZ_d}$"
    rhs = alpha_MZ
    rhs_label = r"$\alpha_{MZ}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    lhs = dd_d_Nd
    lhs_label = r"$\ddot{d}_{N_d}$"
    rhs = dd_d_N
    rhs_label = r"$\ddot{d}_{N}$"
    plot_eq(lhs, lhs_label, rhs, rhs_label)

### Pre-contact controller performance

In [ ]:
plot_eq(v_M[:,2].flatten(), r"$v_{MZ}$", sim.pre_contact_ctrl.v_MZd, r"$v_{MZd}$", t_start_=2, t_end_=4)

## [Inverse Dynamics] Checking decision variable values

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    debug_F_NM = scipy.interpolate.interp1d(d_times, debug['F_NM'], axis=0)(l_times)
    debug_F_FMT = scipy.interpolate.interp1d(d_times, debug['F_FMT'], axis=0)(l_times)
    debug_F_FMX = scipy.interpolate.interp1d(d_times, debug['F_FMX'], axis=0)(l_times)
    debug_F_ContactMY = scipy.interpolate.interp1d(d_times, debug['F_ContactMY'], axis=0)(l_times)
    debug_F_ContactMZ = scipy.interpolate.interp1d(d_times, debug['F_ContactMZ'], axis=0)(l_times)
    debug_F_NL = scipy.interpolate.interp1d(d_times, debug['F_NL'], axis=0)(l_times)
    debug_F_FLT = scipy.interpolate.interp1d(d_times, debug['F_FLT'], axis=0)(l_times)
    debug_F_FLX = scipy.interpolate.interp1d(d_times, debug['F_FLX'], axis=0)(l_times)
    debug_a_MX = scipy.interpolate.interp1d(d_times, debug['a_MX'], axis=0)(l_times)
    debug_a_MT = scipy.interpolate.interp1d(d_times, debug['a_MT'], axis=0)(l_times)
    debug_a_MY = scipy.interpolate.interp1d(d_times, debug['a_MY'], axis=0)(l_times)
    debug_a_MZ = scipy.interpolate.interp1d(d_times, debug['a_MZ'], axis=0)(l_times)
    debug_a_MN = scipy.interpolate.interp1d(d_times, debug['a_MN'], axis=0)(l_times)
    debug_a_LT = scipy.interpolate.interp1d(d_times, debug['a_LT'], axis=0)(l_times)
    debug_a_LN = scipy.interpolate.interp1d(d_times, debug['a_LN'], axis=0)(l_times)
    debug_alpha_MX = scipy.interpolate.interp1d(d_times, debug['alpha_MX'], axis=0)(l_times)
    debug_alpha_MY = scipy.interpolate.interp1d(d_times, debug['alpha_MY'], axis=0)(l_times)
    debug_alpha_MZ = scipy.interpolate.interp1d(d_times, debug['alpha_MZ'], axis=0)(l_times)
    debug_dd_theta_L = scipy.interpolate.interp1d(d_times, debug['dd_theta_L'], axis=0)(l_times)
    debug_dd_d_N = scipy.interpolate.interp1d(d_times, debug['dd_d_N'], axis=0)(l_times)
    debug_dd_d_T = scipy.interpolate.interp1d(d_times, debug['dd_d_T'], axis=0)(l_times)
    debug_ddqs = []
    for i in range(nq):
        debug_ddqs.append(scipy.interpolate.interp1d(d_times, debug['ddq_' + str(i)], axis=0)(l_times))

In [ ]:
if (ctrl_paradigm == plant.simulation.CtrlParadigm.INVERSE_DYNAMICS):
    plot_eq(debug_F_NM, "F_NM (debug)", F_NM, "F_NM (actual)")
    plot_eq(debug_F_FMT, "F_FMT (debug)", F_FMT, "F_FMT (actual)")
    plot_eq(debug_F_FMX, "F_FMX (debug)", F_FMX, "F_FMX (actual)")
    plot_eq(debug_F_ContactMY, "F_ContactMY (debug)", F_ContactMY, "F_ContactMY (actual)")
    plot_eq(debug_F_ContactMZ, "F_ContactMZ (debug)", F_ContactMZ, "F_ContactMZ (actual)")
    plot_eq(debug_F_NL, "F_NL (debug)", F_NL, "F_NL (actual)")
    plot_eq(debug_F_FLT, "F_FLT (debug)", F_FLT, "F_FLT (actual)")
    plot_eq(debug_F_FLX, "F_FLX (debug)", F_FLX, "F_FLX (actual)")
    plot_eq(debug_a_MX, "a_MX (debug)", a_MX, "a_MX (actual)")
    plot_eq(debug_a_MT, "a_MT (debug)", a_MT, "a_MT (actual)")
    plot_eq(debug_a_MY, "a_MY (debug)", a_MY, "a_MY (actual)")
    plot_eq(debug_a_MZ, "a_MZ (debug)", a_MZ, "a_MZ (actual)")
    plot_eq(debug_a_MN, "a_MN (debug)", a_MN, "a_MN (actual)")
    plot_eq(debug_a_LT, "a_LT (debug)", a_LT, "a_LT (actual)")
    plot_eq(debug_a_LN, "a_LN (debug)", a_LN, "a_LN (actual)")
    plot_eq(debug_alpha_MX, "alpha_MX (debug)", alpha_MX, "alpha_MX (actual)")
    plot_eq(debug_alpha_MY, "alpha_MY (debug)", alpha_MY, "alpha_MY (actual)")
    plot_eq(debug_alpha_MZ, "alpha_MZ (debug)", alpha_MZ, "alpha_MZ (actual)")
    plot_eq(debug_dd_theta_L, "dd_theta_L (debug)", dd_theta_L, "dd_theta_L (actual)")
    plot_eq(debug_dd_d_N, "dd_d_N (debug)", dd_d_N, "dd_d_N (actual)")
    plot_eq(debug_dd_d_T, "dd_d_T (debug)", dd_d_T, "dd_d_T (actual)")
    for i in range(nq):
        plot_eq(debug_ddqs[i], "dd_d " + str(i) + " (debug)", dd_q[:,i].flatten(), "dd_d " + str(i) + " (actual)")

## General inspection
Any other plots used for just looking at values in the sim, not directly related to the controller.

### Friction estimates

#### Norm

In [ ]:
lhs = np.linalg.norm(np.expand_dims(F_FLH, [1,2])*H_hat + np.expand_dims(F_FLT, [1,2])*T_hat, axis=1)
lhs_label = r"$||F_{FLT} + F_{FLH}||$"

rhs = np.abs(np.expand_dims(mu*mu_S*F_NL, 1))
rhs_label = "$|\mu\mu_{S}F_{NL}|$"

plot_eq(lhs, lhs_label, rhs, rhs_label)
plt.show()

In [ ]:
lhs = s_S
lhs_label = r"Slip speed candidate norm"

rhs = slip_speed
rhs_label = "Actual slip speed"

plot_eq(lhs, lhs_label, rhs, rhs_label)
plt.show()

#### Direction

In [ ]:
F_hat = np.expand_dims(F_FLT, [1,2])*T_hat + np.expand_dims(F_FLH, [1,2])*H_hat
F_hat = F_hat/np.expand_dims(np.linalg.norm(F_hat, axis=1), 2)

In [ ]:
sig1 = s_hat
sig1_label = r"$\hat s$"

sig2 = F_hat
sig2_label = r"$\hat F$"
plot_vec(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

In [ ]:
sig1 = s_hat
sig1_label = r"$\hat s$"

sig2 = F_hat
sig2_label = r"$\hat F$"
plot_vec_HTN(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

#### Comparison against actual values

In [ ]:
F_mag = mu_S*F_NL*mu

In [ ]:
F_F_est = s_hat*np.expand_dims(np.vstack((F_mag,F_mag,F_mag)).T, 2)

In [ ]:
sig1 = F_F_est
sig1_label = r"Estimated friction"

sig2 = F_contact_L - N_hat_proj_mat@F_contact_L
sig2_label = r"Actual friction"
plot_vec_HTN(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

### Friction measurements

#### Friction cone
Whether or not we bump up against the friction cone.

In [ ]:
lw=5
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end],
         abs(F_FLT[idx_start:idx_end]), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(l_times[idx_start:idx_end],
         abs(F_FMT[idx_start:idx_end]), label=r"$\left|F_{FM}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(l_times[idx_start:idx_end],
         abs(F_NL[idx_start:idx_end])*mu, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')

plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel("Time")
plt.show()

#### Slip speed

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times[idx_start:idx_end], slip_speed[idx_start:idx_end], label="slip speed", linewidth=5)
plt.axhline(constants.v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
# plt.ylim(0, 2*constants.v_stiction)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

### Joint torques

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[0,idx_start:idx_end], linewidth=5, color='r', label=r"\tau_x")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[1,idx_start:idx_end], linewidth=5, color='g', label=r"\tau_y")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[2,idx_start:idx_end], linewidth=5, color='b', label=r"\tau_z")
plt.autoscale(enable=True, axis='x', tight=True)

In [ ]:
lhs = a_LT
lhs_label = r"$a_{LT}$"
rhs = (-w_L/2)*d_theta_L**2
rhs_label = r"$-\frac{w_L}{2}\dot\theta^2$"
plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
lhs = a_LN
lhs_label = r"$a_{LN}$"
rhs = (w_L/2)*dd_theta_L
rhs_label = r"$\frac{w_L}{2}\ddot\theta$"
plot_eq(lhs, lhs_label, rhs, rhs_label)

### Contact normal

In [ ]:
plot_vec(N_hat, r"True $\hat N$", N_hat_geo, r"Geometric $\hat N$")

### Kinematic singularities
How close we are to kinematic singularities, measured in various ways.

#### Singular value decomposition

In [ ]:
min_svds = np.min(np.linalg.svd(J)[1], axis=1)
plot_eq(min_svds, "minimum singular value", 0, "0")

#### Volume of manipulability ellipsoid

Page 195 of Modern Robotics says that the volume of the manipulability ellipsoid is proportional to $\sqrt{\det\left(JJ^T\right)}$.

In [ ]:
V = np.linalg.det(np.matmul(J, J.transpose([0, 2, 1])))

In [ ]:
plot_eq(V, r"$\sqrt{\det\left(JJ^T\right)}$", 0, "0")

#### Mass matrix deteriminant

In [ ]:
plot_eq(np.linalg.det(M), "determinant of the mass matrix", 0, "0")

### Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
# Plot drag forces
nb = sim.plant.num_bodies
vel_traces = []
for b in sim.paper.link_idxs:
    rho = 1.225
    C_d = sim.paper.w_L*sim.paper.x_dim
    vel_sqr = log.data()[sim.log_wrapper.entries_per_body*b+7]**2+log.data()[sim.log_wrapper.entries_per_body*b+8]**2
    f = 0.5*rho*C_d*vel_sqr
    plt.plot(f)

plt.axhline(sim.paper.m_L*9.81, color='black', linestyle='--')
plt.show()

### Plots used in original paper

In [ ]:
# Generate plots for paper
nb = sim.plant.num_bodies()
y_traces = []
z_traces = []
theta_traces = []
for b in sim.paper.link_idxs:
    y_traces.append(log.data()[sim.log_wrapper.entries_per_body*b+1])
    z_traces.append(log.data()[sim.log_wrapper.entries_per_body*b+2])
    theta_traces.append(log.data()[sim.log_wrapper.entries_per_body*b+3])
y_traces = np.array(y_traces)
z_traces = np.array(z_traces)
theta_traces = np.array(theta_traces)

times_ = np.arange(0,effective_tspan, effective_tspan/10)
cmap = cm.get_cmap("viridis_r")
plt.figure(figsize=(2*3,2*2))
for t in times_:
    c = cmap(t/effective_tspan)
    idx = np.argmax(log.sample_times() >= t)

    # Plot paper
    # PROGRAMMING: Account for sim.paper thickness in plots
    for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
        y0 = y - np.cos(theta)*w_L/2
        z0 = z - np.sin(theta)*w_L/2
        y1 = y + np.cos(theta)*w_L/2
        z1 = z + np.sin(theta)*w_L/2
        plt.plot([y0, y1], [z0, z1], color=c)


    # Plot manipulator
    plt.scatter(log.data()[sim.log_wrapper.entries_per_body*sim.contact_body_idx+1,idx], 
                log.data()[sim.log_wrapper.entries_per_body*sim.contact_body_idx+2,idx],
                color=c, s=300, zorder=1)

xlim = plt.xlim()
ylim = plt.ylim()
plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
plt.xlim(xlim)
plt.ylim(ylim)
cb = plt.colorbar()
cb.set_label("Time")
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.show()

# Debugging sandbox
Plots that I don't intend to keep long-term, but are useful to have while debugging my current problem.

## Simplified normal acceleration analysis
### Link vs. manipulator normal acceleration

In [ ]:
plot_eq(a_MN, "a_MN", a_LN, "a_LN")

This confirms we can analyze the two bodies as moving together.

### Adding together N hat equations
Because they're moving together, everything should cancel. First, just adding back the sphere equations:

In [ ]:
# TODO

## Debug constant

In [ ]:
x0s = np.array(sim.fold_ctrl.debug['x0'])

In [ ]:
x0s.shape

In [ ]:
x0_times = np.array(sim.fold_ctrl.debug['times'])

In [ ]:
x0s[0,5]

In [ ]:
p_M[0,2]

In [ ]:
ff_wrenchs = np.array(sim.fold_ctrl.debug['ff_wrench'])

In [ ]:
ff_wrenchs.shape

In [ ]:
np.any(ff_wrenchs[:,0]) or np.any(ff_wrenchs[:,1]) or np.any(ff_wrenchs[:,2])

In [ ]:
N_hat.shape

In [ ]:
colors = ['red','green','blue']
for i in range(3):
    plt.plot(l_times, 10*N_hat[:,i].flatten(), color=colors[i], linestyle='--')
    plt.plot(x0_times, ff_wrenchs[:,3+i].flatten(), color=colors[i], alpha=0.3)
plt.plot(x0_times, np.linalg.norm(ff_wrenchs[:,3:], axis=1), color='black', linestyle='--')

In [ ]:
np.linalg.norm(ff_wrenchs[:,3:], axis=1)

# Panda debuging plots

In [ ]:
t_query = 29.8
idx_query = np.argmax(l_times > t_query)

In [ ]:
print("[")
for q_ in q[idx_query].flatten():
    print("    {},".format(q_))
print("]")

In [ ]:
for i in range(nq):
    plt.figure()
    plt.plot(l_times, joint_centering_torque[:,i], 'o--', linewidth=3, label="Torque")
    plt.plot(l_times, manipulator.starting_q[i]-q[:,i], 'o--', linewidth=3, label="Error")
    plt.plot(l_times, d_q[:,i], 'o--', linewidth=3)
    plt.legend()

In [ ]:
plt.figure()
plt.plot(l_times, d_T/np.nanmax(np.abs(d_T)))
plt.plot(l_times, d_d_T/np.nanmax(np.abs(d_d_T)))
plt.plot(l_times, dd_d_T/np.nanmax(np.abs(dd_d_T)))

In [ ]:
plt.figure()
plt.plot(l_times, p_LE[:,0]/np.nanmax(np.abs(p_C[:,0])), color='r')
plt.plot(l_times, p_LE[:,1]/np.nanmax(np.abs(p_C[:,1])), color='g')
plt.plot(l_times, p_LE[:,2]/np.nanmax(np.abs(p_C[:,2])), color='b')

In [ ]:
plt.figure()
plt.plot(l_times, p_C[:,0]/np.nanmax(np.abs(p_C[:,0])), color='r')
plt.plot(l_times, p_C[:,1]/np.nanmax(np.abs(p_C[:,1])), color='g')
plt.plot(l_times, p_C[:,2]/np.nanmax(np.abs(p_C[:,2])), color='b')

In [ ]:
np.all(np.isnan(d_d_T))